In [ ]:
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [ ]:
# default_exp files

<IPython.core.display.Javascript object>

# File

In [ ]:
# export

import os
import io
import hashlib

from minio import Minio
from pathlib import Path
from typing import Optional
from pydantic import BaseModel


def calculate_checksum(content):
    return hashlib.md5(content).hexdigest()


def checksum_for_path(path):
    with path.open("rb") as f:
        checksum = calculate_checksum(f.read())
    return checksum


def create_via_filesystem(path, size):
    path.parent.mkdir(parents=True, exist_ok=True)
    if not path.exists():
        with path.open("wb") as f:
            f.write(os.urandom(size))
    return checksum_for_path(path)


def get_minio_client_and_bucket(endpoint, params, bucket_name):
    client = Minio(endpoint, **params)
    found = client.bucket_exists(bucket_name)
    if not found:
        client.make_bucket(bucket_name)
    return client


def build_create_via_minio():
    client_endpoint = "127.0.0.1:9000"
    client_params = {
        "access_key": "minioadmin",
        "secret_key": "minioadmin",
        "secure": False,
    }
    client = None
    bucket_name = "wis"

    def create_via_minio(path, size):
        nonlocal client
        if client is None:
            # dont try to connect to minio server until create_via_minio
            # is really used, because it will break CI otherwise
            client = get_minio_client_and_bucket(
                client_endpoint, client_params, bucket_name
            )
        data = os.urandom(size)

        result = client.put_object(
            bucket_name,
            str(path),
            io.BytesIO(data),
            size,
        )
        return calculate_checksum(data)

    return create_via_minio


FILE_CREATORS = {"filesystem": create_via_filesystem, "minio": build_create_via_minio()}


class BenchmarkFile(BaseModel):
    number: int
    base_path: str
    size: int
    data_root: str = "data"
    hostname: str = "localhost"
    port: int = 8000
    checksum: Optional[str] = None
    creator_name: str = "filesystem"

    @property
    def filesystem_path(self):
        return Path(self.data_root) / self.base_path / str(self.number)

    def get_or_create(self):
        create = FILE_CREATORS.get(self.creator_name)
        self.checksum = create(self.filesystem_path, self.size)

    @property
    def path(self):
        return f"{self.data_root}/{self.base_path}/{self.number}"

    @property
    def host(self):
        return f"http://{self.hostname}:{self.port}"

    @property
    def url(self):
        return f"{self.host}/{self.path}"

<IPython.core.display.Javascript object>

## Usage

In [ ]:
base_path = "3000000_2_12500000"
benchmark_file = BenchmarkFile(number=0, size=10 ** 6 * 3, base_path=base_path)
benchmark_file.url

'http://localhost:8000/data/3000000_2_12500000/0'

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

base_path = "3000000_2_12500000"
benchmark_file = BenchmarkFile(
    number=0, size=10 ** 6 * 3, base_path=base_path, creator_name="minio"
)
benchmark_file.get_or_create()
benchmark_file.url

'http://localhost:8000/data/3000000_2_12500000/0'

<IPython.core.display.Javascript object>

In [ ]:
benchmark_file.checksum

'07d28390cf199e7d1079aa7f5ccc7b9d'

<IPython.core.display.Javascript object>

In [ ]:
file_size = 10 ** 6 * 3


def create_file(path, size):
    assert size == file_size
    return "test_md5sum"


FILE_CREATORS["stub_creator"] = create_file


benchmark_file = BenchmarkFile(
    number=0, size=file_size, base_path=base_path, creator_name="stub_creator"
)
benchmark_file.get_or_create()

<IPython.core.display.Javascript object>

In [ ]:
# hide
# dont_test

from nbdev.export import notebook2script

notebook2script()

Converted 00_index.ipynb.
Converted 01_host.ipynb.
Converted 02_file.ipynb.
Converted 03_registry.ipynb.
Converted 04_epochs.ipynb.
Converted 10_servers.ipynb.
Converted 11_views_for_fastapi_server.ipynb.
Converted 12_views_for_django_server.ipynb.
Converted 15_servers_started_locally.ipynb.
Converted 16_servers_started_via_docker.ipynb.
Converted 20_clients.ipynb.
Converted 21_benchmark_client_implementations.ipynb.
Converted 30_control_server.ipynb.
Converted 31_control_client.ipynb.
Converted 32_control_cli.ipynb.
Converted 40_results.ipynb.
Converted 41_repositories.ipynb.
Converted 42_sqlite_repository.ipynb.
Converted 50_benchmark_without_benchmark.ipynb.
Converted 51_benchmark_remote_control_server.ipynb.
Converted 60_legacy.ipynb.
Converted 61_run_benchmark.ipynb.


<IPython.core.display.Javascript object>